## Creating the data set using tapping information for 70000 samples

In [1]:
import synapseclient
import pandas as pd
import json
import csv

In [2]:

def TapPerformance(TapPD1):
    TapPD1.loc[TapPD1.TappedButtonId == 'TappedButtonLeft', 'TappedButtonId'] = 0
    TapPD1.loc[TapPD1.TappedButtonId == 'TappedButtonRight', 'TappedButtonId'] = 1
    TapPD1.loc[TapPD1.TappedButtonId == 'TappedButtonNone', 'TappedButtonId'] = 2
    #TapPD1.TappedButtonId[TapPD1.TappedButtonId == 'TappedButtonLeft'] = 0
    #TapPD1.TappedButtonId[TapPD1.TappedButtonId == 'TappedButtonRight'] = 1
    #TapPD1.TappedButtonId[TapPD1.TappedButtonId == 'TappedButtonNone'] = 2
    TapPD1 = Artifactfixed1(TapPD1)
    TapPerf= TapPD1.shape[0]
    return (TapPerf)

In [3]:
#Index=list()
def Artifactfixed(TapCont):
    Ind =  [True] * TapCont.shape[0]
    for item in range(0,TapCont.shape[0]-1):
        if TapCont.TapCoordinate[item]==TapCont.TapCoordinate[item+1]:
            if TapCont.TappedButtonId[item]==2:
                Ind[item]=False
            elif TapCont.TappedButtonId[item+1]==2:
                Ind[item]=False
    TapContArtFixed=TapCont[Ind]
    return TapContArtFixed
          

In [4]:
def Artifactfixed1(TapCont):
    Ind =  [True] * TapCont.shape[0]
    for item in range(0,TapCont.shape[0]-1):
        if TapCont.TappedButtonId[item]==2:
            Ind[item]=False
    TapContArtFixed=TapCont[Ind]
    return TapContArtFixed

In [5]:
Cred = open("Credentials.txt","r",encoding='utf8')
lines=Cred.readlines()
username1=lines[0][:-1]
password1=lines[1][:-1]
Cred.close()



In [6]:
syn = synapseclient.Synapse()
syn.login(username1, password1)


## QUERY THE mPower PROJECT (syn4993293) FOR ALL OF THE TABLES
# See examples: https://python-docs.synapse.org/build/html/index.html 
# and https://www.synapse.org/#!Synapse:syn4993293/discussion/threadId=6785
with open('TappingTable.csv') as csvfile:
    TapTable=pd.read_csv(csvfile, delimiter=',')
with open('DemoTable.csv') as csvfile:
    DemoTable=pd.read_csv(csvfile, delimiter=',') 
#Using Patient column (healthCode) as index for the Data frames
#TapTableB = TapTable.set_index("healthCode", drop = False) 
#DemoTableB = DemoTable.set_index("healthCode", drop = False)    

      
#TapPD = list()
PathList = list()
LabelList = list()
#PatientType = list()
#PatientList = list()
#AgeList = list()
Perform =list()
for offset in range(0,70000,100):
    results = syn.tableQuery('SELECT * FROM syn5511439 LIMIT 100 OFFSET '+str(offset))
    file_map=syn.downloadTableColumns(results,['tapping_results.json.TappingSamples'])
    for file_handle_id, path in file_map.items():
        #TapPD.append(pd.read_json(path))
        PathList.append(path)
        LabelList.append(int(file_handle_id))
        #PatientType.append(TapTableB['medTimepoint'][TapTable['tapping_results.json.TappingSamples'] == int(file_handle_id)])
        Perform.append(TapPerformance(pd.read_json(path))) #Tap performance
        #PatientList.append(TapTableB['healthCode'][TapTableB['tapping_results.json.TappingSamples'] == int(file_handle_id)])
        #DemoTableB.loc[(TapTableB['healthCode'][TapTableB['tapping_results.json.TappingSamples'] == int(file_handle_id)]),"age"]
        

Welcome, Catalina Rivera!



[WARNING] /Users/catalina/anaconda/envs/insight/lib/python3.8/site-packages/synapseclient/client.py:3481: UserWarning: Weird file handle: None
  warnings.warn("Weird file handle: %s" % file_handle_id)



[WARNING] /Users/catalina/anaconda/envs/insight/lib/python3.8/site-packages/synapseclient/client.py:3481: UserWarning: Weird file handle: None
  warnings.warn("Weird file handle: %s" % file_handle_id)



[WARNING] /Users/catalina/anaconda/envs/insight/lib/python3.8/site-packages/synapseclient/client.py:3481: UserWarning: Weird file handle: None
  warnings.warn("Weird file handle: %s" % file_handle_id)



[WARNING] /Users/catalina/anaconda/envs/insight/lib/python3.8/site-packages/synapseclient/client.py:3481: UserWarning: Weird file handle: None
  warnings.warn("Weird file handle: %s" % file_handle_id)



[WARNING] /Users/catalina/anaconda/envs/insight/lib/python3.8/site-packages/synapseclient/client.py:3481: UserWarning: Weird file handle: None
  warnings.warn("Weird file handle: %s" % file_handle_id)



[WARNING] /Users/catalina/anaconda/envs/insight/lib/python3.8/site-packages/synapseclient/client.py:3481: UserWarning: Weird file handle: None
  warnings.warn("Weird file handle: %s" % file_handle_id)



[WARNING] /Users/catalina/anaconda/envs/insight/lib/python3.8/site-packages/synapseclient/client.py:3481: UserWarning: Weird file handle: None
  warnings.warn("Weird file handle: %s" % file_handle_id)



In [7]:
TapTable.columns
#Drop columns no needed
#TapTable=TapTable.drop(['ROW_ID', 'ROW_VERSION','recordId','createdOn','appVersion','phoneInfo','accel_tapping.json.items','tapping_results.json.ButtonRectLeft','tapping_results.json.ButtonRectRight','tapping_results.json.endDate','tapping_results.json.startDate','tapping_results.json.TappingViewSize','medTimepoint'], axis=1)
#Creating directory
d = {'tapping_results.json.TappingSamples':LabelList,'TapPerform':Perform}
PerfInfo = pd.DataFrame(d)
#Merges TapTables with the PerfInfo and drops thos values in TapTable that are not defined in PerInfo
#TapTable=TapTable.merge(PerfInfo,left_on='tapping_results.json.TappingSamples', right_on='tapping_results.json.TappingSamples')
TapTable=TapTable.merge(PerfInfo)



In [8]:
#Save to a csv file
TapTable.head()
TapTable.to_csv(r'/Users/catalina/Dropbox/Insight/Parkinson Disease/TappingTableWithPerform.csv')


In [9]:
TapTable.shape

(69993, 16)

In [10]:
DemoTable.shape

(6805, 33)